1\. Write a function that converts number representation (bin<->dec<->hex)

In [4]:
# number is entered as a string!

# this version uses builtin functions
def cnv(from_base, to_base, number):
    """ Handles base conversions """
    if from_base == 'd':
        number = int(number)
    elif from_base == 'b':
        number = int(number,2)
    elif from_base == 'h':
        number = int(number,16)
        
    if to_base == 'b':
        number = bin(number)
    elif to_base == 'h':
        number = hex(number)
    return str(number)

def cnv(from_base, to_base, number):
    """ Handles base conversions """
    if from_base == 'd':
        if to_base == 'b':
            return db(number)
        elif to_base == 'h':
            return dh(number)

    elif from_base == 'b':
        if to_base == 'd':
            return bd(number)
        elif to_base == 'h':
            return dh(bd(number))

    elif from_base == 'h':
        if to_base == 'b':
            return db(hd(number))
        elif to_base == 'd':
            return hd(number)
    return str(number)


def db(number):
    """ DEC to BIN string conversion """
    integ = int(number)
    out = ''
    while integ != 0:
        rem = integ % 2 # remainder
        out = str(rem) + out
        integ //= 2 # integer division
    return out

def dh(number):
    """ DEC to HEX string conversion """
    integ = int(number)
    out = ''
    while integ != 0:
        rem = integ % 16 # remainder
        if rem == 10:
            out = 'A' + out
        elif rem == 11:
            out = 'B' + out
        elif rem == 12:
            out = 'C' + out
        elif rem == 13:
            out = 'D' + out
        elif rem == 14:
            out = 'E' + out
        elif rem == 15:
            out = 'F' + out
        else:
            out = str(rem) + out # 0-9 fallback
        integ //= 16 # integer division
    return out

def bd(number):
    """ BIN to DEC string conversion """
    out = 0
    expo = len(number)-1 # MSB exponent
    for c in number:
        out += int(c) * 2**expo
        expo -= 1
    return out

def hd(number):
    """ HEX to DEC string conversion """
    out = 0
    expo = len(number)-1 # MSB exponent
    for c in number:
        try:
            char = int(c)
        except ValueError:
            # conversion of a letter: substitute instead
            if c == 'A':
                char = 10
            elif c == 'B':
                char = 11
            elif c == 'C':
                char = 12
            elif c == 'D':
                char = 13
            elif c == 'E':
                char = 14
            elif c == 'F':
                char = 15
        out += char * 16**expo
        expo -= 1
    return out

print(cnv('d','d','15'))
print(cnv('d','b','15'))
print(cnv('d','h','15'))

print(cnv('b','d','1111'))
print(cnv('b','b','1111'))
print(cnv('b','h','1111'))

print(cnv('h','d','F'))
print(cnv('h','b','F'))
print(cnv('h','h','F'))

15
1111
F
15
1111
F
15
1111
F


2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent)

In [5]:
def single_precision(word):
    if len(word) > 32:
        word = word[:32] # crop
    elif len(word) < 32:
        word = word.zfill(32) # zero-padding
    
    sgn = str( (-1)**int(word[0]) )
    exp = str( bd(word[1:9])-127 )
    
    man = 1
    expo = 1
    for c in binword[9:]:
        man += int(c) * 2**(-expo)
        expo += 1
    man = str(man)
    return sgn,exp,man

binword = '11000011011010010111101101101111'
s,e,m = single_precision(binword)
print("Word: {}".format(binword))
print("S: {}   E: {}   M: {}".format(s,e,m))
# note that mantissa here includes 1. prefix while it should not

Word: 11000011011010010111101101101111
S: -1   E: 7   M: 1.8240793943405151


3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tips**: define two variables inizialized to 1 and halve/double them enough time to exceed the under/over-flow limits  

In [6]:
over = 1.0
under = 1.0

while True:
    temp = over * 2
    if abs(temp) != float('inf'):
        over = temp
    else:
        print(temp) # should be inf
        print("Overflow value: {}".format(over))
        break

while True:
    temp = under / 2
    if temp != 0:
        under = temp
    else:
        print(temp) # should be 0.0
        print("Underflow value: {}".format(under))
        break

inf
Overflow value: 8.98846567431158e+307
0.0
Underflow value: 5e-324


4\. Write a program to determine the machine precision

**Tips**: define a new variable by adding a smaller and smaller value (proceeding similarly to prob. 2) to an original variable and check the point where the two are the same 

In [7]:
a = 1.
b = 1.
qt = .5

while a+qt != b:
    qt /= 2
print(qt*2) # one step back

# compare to true result:
import numpy as np
np.finfo(float).eps

2.220446049250313e-16


2.220446049250313e-16

5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying top and bottom by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what previously obtained? Why?

**FR:** Analytical computation shows the real roots are $-1e\pm 6$. The problem is we are multiplying very small $a$,$c$ with a relatively big $b$ and this causes floating point arithmetic errors: standard formula gives us a good root and one that is not exact, reversing the formula gives us inverted results, so the best strategy seems to use both methods and respectively save only the exact roots. However, these roots are always (acceptable) approximations of real values.

(c) write a function that compute the roots of a quadratic equation accurately in all cases

In [8]:
from math import sqrt

# this returns one bad and one good root
def quad1(a,b,c):
    d = sqrt(b**2 - 4*a*c)
    return ( (-b+d)/(2*a), (-b-d)/(2*a) )

# this returns one good and one bad root
def quad2(a,b,c):
    d = sqrt(b**2 - 4*a*c)
    return ( (2*c)/(-b-d), (2*c)/(-b+d) )

# combine previous to keep only good solutions
def quad(a,b,c):
    x = quad1(a,b,c)
    y = quad2(a,b,c)
    return (x[1],y[0])

a = 0.001
b =  1000
c = 0.001
print(quad1(a,b,c))
print(quad2(a,b,c))
print(quad(a,b,c))

(-9.999894245993346e-07, -999999.999999)
(-1.000000000001e-06, -1000010.5755125057)
(-999999.999999, -1.000000000001e-06)


6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare with the answer your program gives. The two will not agree perfectly. Why not?

**FR:** The exact result is $1$. Again, considered $\delta$ is a small number, compared to $1$ and we have the same type of error as above.

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

**FR:** Accuracy has a peak for a $\delta$ that lies in the middle of the considered interval: if we plot the values, the resulting shape is convex on this minima point.

In [9]:
fun = lambda x: x*(x-1)
def deriv(point, delta):
    return (fun(point+delta) - fun(point)) / delta

for delta in range(2,16,2):
    res = deriv(1.0, 10**(-delta))
    print("delta = {:2}   res = {:.20f}   abs diff = {}".format(delta, res, abs(res-1)))


delta =  2   res = 1.01000000000000089706   abs diff = 0.010000000000000897
delta =  4   res = 1.00009999999988985486   abs diff = 9.999999988985486e-05
delta =  6   res = 1.00000099991773327979   abs diff = 9.99917733279787e-07
delta =  8   res = 1.00000000392252874626   abs diff = 3.922528746258536e-09
delta = 10   res = 1.00000008284037100736   abs diff = 8.284037100736441e-08
delta = 12   res = 1.00008890058334132256   abs diff = 8.890058334132256e-05
delta = 14   res = 0.99920072216265087839   abs diff = 0.0007992778373491216


7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which it's known to be $I=\frac{\pi}{2}=1.57079632679...$.
Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a program to compute the integral with $N=100$. How does the result compares to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute?

**FR:** The correct value of $N$ depends heavily on the configuration of the machine this script is running on. I included as comments the values obtained in the lab.

In [10]:
from math import sqrt

fun = lambda x: sqrt(1-x**2)
def intgrl(N):
    mn = -1.0 # min x value
    mx = 1.0  # max x value
    step = (mx-mn)/N
    
    x = mn
    res = 0.0
    while x <= mx:
        res += 2/N * fun(x)
        x += step
    return res

real_result = 1.57079632679 # circa
res = intgrl(100)
print("result = {}   abs diff = {}".format(res, abs(res-real_result)))

# second part, by trials
#%time intgrl(100000)  # takes circa 25ms
#%time intgrl(1000000) # takes circa 254ms
#%time intgrl(4000000) # takes circa 993ms: roughly this is the required N for this machine
#%time intgrl(5000000) # takes circa 1.24s
# N for 1-minute execution was not tested in the lab

result = 1.5691342555492493   abs diff = 0.0016620712407506844
